In [2]:
import numpy as np
import cupy as cp
from cupy.random import randint as RandInt
print("numpy", np.__version__)
print("cupy", cp.__version__)

numpy 1.18.1
cupy 7.2.0


In [3]:
cp.cuda.Device(1).use()

In [4]:
num = 100_000
x = RandInt(1, high=(4096*4096)-1, size=num)
y = RandInt(1, high=(4096*4096)-1, size=num)
bool_arr = cp.zeros(num, dtype=bool)
X = cp.asnumpy(x)
Y = cp.asnumpy(y)
#print(x)
#print(y)
%timeit -n 1 np.isin(X,Y)
print(x.dtype)
kernel = cp.ElementwiseKernel('T num,  T x,  raw T y', 'bool z',
    '''int t = 0; 
    z = 0;
    #pragma unroll
    for(t = 0; t < num; t++) z = z || (x == y[t]);''',
    'my_kernel')

16.5 ms ± 941 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
int64


In [11]:
%timeit -n 1 bool_arr = kernel(num, x, y)

The slowest run took 7.71 times longer than the fastest. This could mean that an intermediate result is being cached.
154 µs ± 97.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit -n 10 bool_arr = kernel(num, x, y)

The slowest run took 149.22 times longer than the fastest. This could mean that an intermediate result is being cached.
291 µs ± 680 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit -n 100 bool_arr = kernel(num, x, y)

The slowest run took 1551.94 times longer than the fastest. This could mean that an intermediate result is being cached.
11.3 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
